# Plot the MCMC results

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 13})

In [ ]:
PARAMETER_NAMES = [
    r'$B_G\;[\mathrm{GeV}^{-2}]$',
    r'$\alpha_{\rm shadowing}$',
    r'$y_{{\rm loss},2}$',
    r'$y_{{\rm loss},4}$',
    r'$y_{{\rm loss},6}$',
    r'$\sigma_{y_{\rm loss}}$',
    r'$\alpha_{\rm rem}$',
    r'$\lambda_B$',
    r'$\sigma_x^{\rm string}\;[{\rm fm}]$',
    r'$\sigma_\eta^{\rm string}$',
    r'$\alpha_{{\rm string}\;{\rm tilt}}$',
    r'$\alpha_{\rm preFlow}$',
    r'$\eta_0$',
    r'$\eta_2$',
    r'$\eta_4$',
    r'$\zeta_{\rm max}$',
    r'$T_{\zeta,0}\;[{\rm GeV}]$',
    r'$\sigma_{\zeta,+}\;[{\rm GeV}]$',
    r'$\sigma_{\zeta,-}\;[{\rm GeV}]$',
    r'$e_{\rm sw}\;[{\rm GeV}/{\rm fm}^3]$'
    ]

prior_ranges = [
    (1,25),
    (0,1),
    (0,2),
    (1,3),
    (1,4),
    (0.1,0.8),
    (0,1),
    (0,1),
    (0.1,0.8),
    (0.1,1),
    (0,1),
    (0,2),
    (0.001,0.3),
    (0.001,0.3),
    (0.001,0.3),
    (0,0.2),
    (0.15,0.25),
    (0.01,0.15),
    (0.005,0.1),
    (0.15,0.5)
] 

def read_pkl_file_chain(PATH_pklfile_chains):
    with open(PATH_pklfile_chains, 'rb') as pf:
        data = pickle.load(pf)
    
    chain = data['chain']

    n_walkers = chain.shape[0]
    n_steps = chain.shape[1]
    n_parameters = chain.shape[2]

    return n_walkers, n_steps, n_parameters, data['chain']

def read_real_parameters(file):
    with open(file, "rb") as pf:
        data = pickle.load(pf)
    return data['1099']['parameter']

def read_real_parameters1(file):
    with open(file, "rb") as pf:
        data = pickle.load(pf)
    return data['1097']['parameter']


In [ ]:
def plot_chain_histogram(n_walkers, n_steps, n_parameters, data_array, filename, true_values=None):
    num_columns = 3

    # Calculate the number of rows needed based on the number of parameters and columns
    num_rows = (n_parameters + num_columns - 1) // num_columns

    # Define the height of each subplot
    subplot_height = 1.25

    # Calculate the total height of the figure
    total_height = subplot_height * num_rows

    fig, axs = plt.subplots(num_rows, num_columns, figsize=(10, total_height), sharex=False)
    axs = axs.flatten()

    for param_index in range(n_parameters):
        # Create a 2D histogram
        concatenated_data = data_array[:, :, param_index].reshape(-1)
        hist, xedges, yedges = np.histogram2d(np.tile(np.arange(n_steps), n_walkers), concatenated_data.flatten(), bins=[n_steps, 30])
        hist = np.ma.masked_where(hist == 0, hist)  # Mask empty bins
        im = axs[param_index].pcolormesh(xedges, yedges, hist.T, cmap='Greens')

        if true_values is not None:
            axs[param_index].axhline(true_values[param_index], color='k', linestyle='--', alpha=0.5)

        axs[param_index].set_ylabel(PARAMETER_NAMES[param_index])
        axs[param_index].set_xlim([0., max(xedges)])
        axs[param_index].set_ylim([prior_ranges[param_index][0], prior_ranges[param_index][1]])

    # Specify indices for which to display x-axis labels
    indices_with_labels = [17, 18, 19]

    # Set x-axis labels and numbers for specified indices
    for i, ax in enumerate(axs):
        if i in indices_with_labels:
            ax.set_xlabel('Steps')
        else:
            ax.set_xticklabels([])

    # Hide the empty subplots
    for i in range(n_parameters, num_columns * num_rows):
        fig.delaxes(axs[i])

    plt.subplots_adjust(wspace=0)
    plt.tight_layout()
    plt.savefig(f"{filename}.pdf")

In [ ]:
def plot_corner_1dataset(n_parameters,data_array,real_parameters,filename):
    samples = data_array[:,:,:].reshape((-1,n_parameters))

    # Create subplots
    fig, axes = plt.subplots(n_parameters, n_parameters, figsize=(25, 25), sharex='col', sharey='row')

    # Loop over each parameter pair
    for i in range(n_parameters):
        for j in range(n_parameters):
            ax = axes[i, j]

            if i == j:
                ax = fig.add_subplot(n_parameters, n_parameters, i * n_parameters + j + 1)
                ax.hist(samples[:, i], bins=20, color='g', histtype='step', density=True)
                ax.set_yticks([])
                ax.set_yticklabels([])
                if i != n_parameters-1:
                    ax.set_xticks([])
                    ax.set_xticklabels([])
                if i == n_parameters-1:
                    ax.tick_params(axis='x', rotation=45, labelsize=10)

                # Calculate percentiles
                percentiles = np.percentile(samples[:, i], [16, 50, 84])
                median = percentiles[1]
                upper = percentiles[2] - median
                lower = median - percentiles[0]

                ax.annotate(fr'${median:.2f}_{{-{lower:.2f}}}^{{+{upper:.2f}}}$', xy=(0.5, 1.05), xycoords='axes fraction', ha='center')

                # Add a vertical line representing the real value
                real_value = real_parameters[i]
                ax.axvline(x=real_value, color='k', linestyle='-')

                ax.set_xlim([prior_ranges[i][0], prior_ranges[i][1]])
            # Only fill the lower triangle
            if i > j:
                ax.hist2d(samples[:, j], samples[:, i], bins=20, cmap='Greens')


            # Set labels on the lowest and leftmost plots
            if i == n_parameters - 1:
                ax.set_xlabel(PARAMETER_NAMES[j])
            if j == 0:
                ax.set_ylabel(PARAMETER_NAMES[i])

            if i < j:
                ax.axis('off')

    # Remove space between subplots
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

    # Remove ticks and labels for the first and last plots
    axes[0, 0].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    axes[n_parameters - 1, n_parameters - 1].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

    # Rotate x-axis tick labels
    for ax in axes[-1]:
        ax.tick_params(axis='x', rotation=45, labelsize=10)

    # Rotate y-axis tick labels
    for ax in axes[:, 0]:
        ax.tick_params(axis='y', rotation=45, labelsize=10)

    plt.savefig(f"{filename}.pdf")

In [ ]:
def plot_corner_compare_datasets(n_parameters,data_array1,data_array2,real_parameters,filename):
    samples1 = data_array1[:,:,:].reshape((-1,n_parameters))
    samples2 = data_array2[:,:,:].reshape((-1,n_parameters))

    # Create subplots
    fig, axes = plt.subplots(n_parameters, n_parameters, figsize=(25, 25), sharex='col', sharey='row')

    # Loop over each parameter pair
    for i in range(n_parameters):
        for j in range(n_parameters):
            ax = axes[i, j]

            if i == j:
                ax = fig.add_subplot(n_parameters, n_parameters, i * n_parameters + j + 1)
                ax.hist(samples1[:, i], bins=20, color='g', histtype='step', density=True)
                ax.hist(samples2[:, i], bins=20, color='magenta', histtype='step', density=True)
                ax.set_yticks([])
                ax.set_yticklabels([])
                if i != n_parameters-1:
                    ax.set_xticks([])
                    ax.set_xticklabels([])
                if i == n_parameters-1:
                    ax.tick_params(axis='x', rotation=45, labelsize=10)

                # Calculate percentiles
                percentiles1 = np.percentile(samples1[:, i], [16, 50, 84])
                median1 = percentiles1[1]
                upper1 = percentiles1[2] - median1
                lower1 = median1 - percentiles1[0]
                axes[0,j].annotate(fr'${median1:.2f}_{{-{lower1:.2f}}}^{{+{upper1:.2f}}}$', xy=(0.5, 1.05), xycoords='axes fraction', ha='center',color='g')

                percentiles2 = np.percentile(samples2[:, i], [16, 50, 84])
                median2 = percentiles2[1]
                upper2 = percentiles2[2] - median2
                lower2 = median2 - percentiles2[0]
                axes[0,j].annotate(fr'${median2:.2f}_{{-{lower2:.2f}}}^{{+{upper2:.2f}}}$', xy=(0.5, 1.25), xycoords='axes fraction', ha='center',color='magenta')

                # Add a vertical line representing the real value
                real_value = real_parameters[i]
                ax.axvline(x=real_value, color='k', linestyle='-')

                ax.set_xlim([prior_ranges[i][0], prior_ranges[i][1]])
            # Only fill the lower triangle
            if i > j:
                ax.hist2d(samples1[:, j], samples1[:, i], bins=20, cmap='Greens')
            if i < j:
                ax.hist2d(samples2[:, j], samples2[:, i], bins=20, cmap='RdPu')

            # Set labels on the lowest and leftmost plots
            if i == n_parameters - 1:
                ax.set_xlabel(PARAMETER_NAMES[j])
            if j == 0:
                if i == 0:
                    ax.set_ylabel(PARAMETER_NAMES[i],labelpad=25)
                else:
                    ax.set_ylabel(PARAMETER_NAMES[i])

    # Remove space between subplots
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

    # Remove ticks and labels for the first and last plots
    #axes[0, 0].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    axes[n_parameters - 1, n_parameters - 1].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

    # Rotate x-axis tick labels
    for ax in axes[-1]:
        ax.tick_params(axis='x', rotation=45, labelsize=10)

    # Rotate y-axis tick labels
    for ax in axes[:, 0]:
        ax.tick_params(axis='y', rotation=45, labelsize=10)

    plt.savefig(f"{filename}.pdf")

In [ ]:
def plot_compare_posteriors_3datasets(n_parameters,data_array1,data_array2,data_array3,real_parameters,dataset_label,filename):
    samples1 = data_array1[:,:,:].reshape((-1,n_parameters))
    samples2 = data_array2[:,:,:].reshape((-1,n_parameters))
    samples3 = data_array3[:,:,:].reshape((-1,n_parameters))

    # Create subplots
    nrows = 4
    ncols = 5
    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 12))

    # Loop over each parameter pair
    for r in range(nrows):
        for c in range(ncols):
            ax = axes[r, c]

            # Calculate the parameter index within the bounds of n_parameters
            param_index = r * ncols + c
            if param_index < n_parameters:
                ax.hist(samples1[:, param_index], bins='auto', color='g', histtype='step', density=True, label=dataset_label[0])
                ax.hist(samples2[:, param_index], bins='auto', color='magenta', histtype='step', density=True, label=dataset_label[1])
                ax.hist(samples3[:, param_index], bins='auto', color='blue', histtype='step', density=True, label=dataset_label[2])

                # Calculate percentiles and annotate
                percentiles1 = np.percentile(samples1[:, param_index], [5, 50, 95])
                median1, lower1, upper1 = percentiles1[1], percentiles1[1]-percentiles1[0], percentiles1[2] - percentiles1[1]
                ax.annotate(fr'${median1:.2f}_{{-{lower1:.2f}}}^{{+{upper1:.2f}}}$', xy=(0.1, 1.05), xycoords='axes fraction', ha='center', color='g', fontsize=12)

                percentiles2 = np.percentile(samples2[:, param_index], [5, 50, 95])
                median2, lower2, upper2 = percentiles2[1], percentiles2[1]-percentiles2[0], percentiles2[2] - percentiles2[1]
                ax.annotate(fr'${median2:.2f}_{{-{lower2:.2f}}}^{{+{upper2:.2f}}}$', xy=(0.5, 1.05), xycoords='axes fraction', ha='center', color='magenta', fontsize=12)

                percentiles3 = np.percentile(samples3[:, param_index], [5, 50, 95])
                median3, lower3, upper3 = percentiles3[1], percentiles3[1]-percentiles3[0], percentiles3[2] - percentiles3[1]
                ax.annotate(fr'${median3:.2f}_{{-{lower3:.2f}}}^{{+{upper3:.2f}}}$', xy=(0.9, 1.05), xycoords='axes fraction', ha='center', color='blue', fontsize=12)

                # Add a vertical line representing the real value
                real_value = real_parameters[param_index]
                ax.axvline(x=real_value, color='k', linestyle='-', label='True value')

                ax.set_xlabel(PARAMETER_NAMES[param_index])

                if param_index == 0:
                    handles, labels = ax.get_legend_handles_labels()
                    #ax.legend(borderaxespad=0.1, fontsize=12)
            else:
                ax.axis('off')  # Turn off empty subplots

            # Remove ticks on y-axis
            ax.set_yscale('log')
            ax.yaxis.set_ticks([])

    # Create legend for the first subplot
    axes[0, 0].legend(handles[:2], labels[:2], borderaxespad=0.1, fontsize=12)
    # Create legend for the second subplot
    axes[0, 1].legend(handles[2:], labels[2:], borderaxespad=0.1, fontsize=12)


    plt.subplots_adjust(hspace=0.4, wspace=0.05)
    plt.tight_layout()
    plt.savefig(f"{filename}.pdf")

In [ ]:
def extract_parameters(data_array):
    samples = data_array[:,:,:].reshape((-1,data_array.shape[-1]))

    for param_index in range(samples.shape[-1]):
        percentiles1 = np.percentile(samples[:, param_index], [5, 50, 95])
        median1, lower1, upper1 = percentiles1[1], percentiles1[1]-percentiles1[0], percentiles1[2] - percentiles1[1]
        print(f"{PARAMETER_NAMES[param_index]}: {median1:.3f}-{lower1:.3f}+{upper1:.3f}")

## Plot chains of the different runs

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
real_parameters1 = read_real_parameters1('./separate_training_posterior_data_1095/example_data_test_point1097.pkl')

PATH_pklfile = './mcmc/mcmc1/chain.pkl'
RUN1_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN1_DATA[0],RUN1_DATA[1],RUN1_DATA[2],RUN1_DATA[3],'chain_RUN1',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc2/chain.pkl'
RUN2_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN2_DATA[0],RUN2_DATA[1],RUN2_DATA[2],RUN2_DATA[3],'chain_RUN2',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc3/chain.pkl'
RUN3_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN3_DATA[0],RUN3_DATA[1],RUN3_DATA[2],RUN3_DATA[3],'chain_RUN3',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc5_morePoints/chain.pkl'
RUN5_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN5_DATA[0],RUN5_DATA[1],RUN5_DATA[2],RUN5_DATA[3],'chain_RUN5',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc5_moreT_moreP/chain.pkl'
RUN5_MORE_TP_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN5_MORE_TP_DATA[0],RUN5_MORE_TP_DATA[1],RUN5_MORE_TP_DATA[2],RUN5_MORE_TP_DATA[3],'chain_RUN5_moreT_moreP',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc6/chain.pkl'
RUN6_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN6_DATA[0],RUN6_DATA[1],RUN6_DATA[2],RUN6_DATA[3],'chain_RUN6',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc7/chain.pkl'
RUN7_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN7_DATA[0],RUN7_DATA[1],RUN7_DATA[2],RUN7_DATA[3],'chain_RUN7',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc8/chain.pkl'
RUN8_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN8_DATA[0],RUN8_DATA[1],RUN8_DATA[2],RUN8_DATA[3],'chain_RUN8',true_values=real_parameters)

PATH_pklfile = './mcmc/mcmc5_1/chain.pkl'
RUN5_1_DATA = read_pkl_file_chain(PATH_pklfile)
plot_chain_histogram(RUN5_1_DATA[0],RUN5_1_DATA[1],RUN5_1_DATA[2],RUN5_1_DATA[3],'chain_RUN5_1097',true_values=real_parameters1)


## Plot the corner plot for each run

In [ ]:
plot_corner_1dataset(RUN1_DATA[2],RUN1_DATA[3],real_parameters,'corner_RUN1')
plot_corner_1dataset(RUN2_DATA[2],RUN2_DATA[3],real_parameters,'corner_RUN2')
plot_corner_1dataset(RUN3_DATA[2],RUN3_DATA[3],real_parameters,'corner_RUN3')
plot_corner_1dataset(RUN5_DATA[2],RUN5_DATA[3],real_parameters,'corner_RUN5')
plot_corner_1dataset(RUN5_MORE_TP_DATA[2],RUN5_MORE_TP_DATA[3],real_parameters,'corner_RUN5_moreT_moreP')
plot_corner_1dataset(RUN6_DATA[2],RUN6_DATA[3],real_parameters,'corner_RUN6')
plot_corner_1dataset(RUN7_DATA[2],RUN7_DATA[3],real_parameters,'corner_RUN7')
plot_corner_1dataset(RUN8_DATA[2],RUN8_DATA[3],real_parameters,'corner_RUN8')
plot_corner_1dataset(RUN5_1_DATA[2],RUN5_1_DATA[3],real_parameters1,'corner_RUN5_1097')

## Compare the posterior curves

In [ ]:
plot_compare_posteriors_3datasets(RUN1_DATA[2],RUN1_DATA[3],RUN2_DATA[3],RUN3_DATA[3],real_parameters, ['PCSK','PCGP','Scikit GP'],'compare_posteriors_RUN1_RUN2_RUN3')

In [ ]:
plot_corner_compare_datasets(RUN1_DATA[2],RUN1_DATA[3],RUN5_MORE_TP_DATA[3],real_parameters,'corner_RUN1_RUN5')

In [ ]:
plot_corner_compare_datasets(RUN1_DATA[2],RUN1_DATA[3],RUN6_DATA[3],real_parameters,'compare_RUN1_RUN6')

In [ ]:
plot_corner_compare_datasets(RUN1_DATA[2],RUN1_DATA[3],RUN7_DATA[3],real_parameters,'compare_RUN1_RUN7')

In [ ]:
plot_corner_compare_datasets(RUN1_DATA[2],RUN1_DATA[3],RUN8_DATA[3],real_parameters,'compare_RUN1_RUN8')

## Compute $\Delta_d$

This is defined as:
$$ \Delta \equiv \frac{1}{N_{\rm param.}}\int \left|\frac{\theta-\theta_{\rm truth}}{\theta_{\rm max}-\theta_{\rm min}}\right|^2 p(\theta)\;\mathrm{d}\theta $$
where $\theta_{\rm truth}$ is the closure test point, $|\theta_{\rm max}-\theta_{\rm min}|$ is the allowed prior range and $p(\theta)$ is the marginalized posterior distribution, where the closure test point is not included.

In [ ]:
def compute_Delta_d(prior_ranges, true_parameters, chain_data):
    samples = chain_data[:,:,:].reshape((-1, chain_data.shape[2]))

    # compute the prior volume
    prior_widths = np.array(prior_ranges)[:,1] - np.array(prior_ranges)[:,0]

    n_steps = samples.shape[0]
    sum_value = 0.
    for i in range(n_steps):
        sum_value += np.sum(((samples[i] - true_parameters) / prior_widths)**2)

    sum_value /= n_steps
    Delta_d = sum_value / chain_data.shape[2]

    return Delta_d

def create_Delta_d_distribution(prior_ranges, true_parameters, chain_data):
    samples = chain_data[:,:,:].reshape((-1, chain_data.shape[2]))

    # compute the prior volume
    prior_widths = np.array(prior_ranges)[:,1] - np.array(prior_ranges)[:,0]

    n_steps = samples.shape[0]
    contribution = []
    for i in range(n_steps):
        contribution.append(np.sum(((samples[i] - true_parameters) / prior_widths)**2)  / chain_data.shape[2])
    
    return contribution


In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc1/chain.pkl'
RUN1_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN1_DATA[3])
print(f"Delta_d for RUN1: {delta_d2}")

delta_1_dist = create_Delta_d_distribution(prior_ranges,real_parameters,RUN1_DATA[3])

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc2/chain.pkl'
RUN2_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN2_DATA[3])
print(f"Delta_d for RUN2: {delta_d2}")

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc3/chain.pkl'
RUN3_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN3_DATA[3])
print(f"Delta_d for RUN3: {delta_d2}")

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc5_morePoints/chain.pkl'
RUN5_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN5_DATA[3])
print(f"Delta_d for RUN5: {delta_d2}")

delta_5_dist = create_Delta_d_distribution(prior_ranges,real_parameters,RUN5_DATA[3])

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc5_moreT/chain.pkl'
RUN5_MORE_T_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN5_MORE_T_DATA[3])
print(f"Delta_d for RUN5: {delta_d2}")

delta_5_dist_moreT = create_Delta_d_distribution(prior_ranges,real_parameters,RUN5_MORE_T_DATA[3])

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc5_moreT_moreP/chain.pkl'
RUN5_MORE_TP_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN5_MORE_TP_DATA[3])
print(f"Delta_d for RUN5: {delta_d2}")

delta_5_dist_moreTP = create_Delta_d_distribution(prior_ranges,real_parameters,RUN5_MORE_TP_DATA[3])

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc6/chain.pkl'
RUN6_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN6_DATA[3])
print(f"Delta_d for RUN6: {delta_d2}")

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc7/chain.pkl'
RUN7_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN7_DATA[3])
print(f"Delta_d for RUN7: {delta_d2}")

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
PATH_pklfile = './mcmc/mcmc8/chain.pkl'
RUN8_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters,RUN8_DATA[3])
print(f"Delta_d for RUN8: {delta_d2}")

delta_8_dist = create_Delta_d_distribution(prior_ranges,real_parameters,RUN8_DATA[3])

In [ ]:
real_parameters1 = read_real_parameters1('./separate_training_posterior_data_1095/example_data_test_point1097.pkl')
PATH_pklfile = './mcmc/mcmc5_1/chain.pkl'
RUN5_DATA = read_pkl_file_chain(PATH_pklfile)

delta_d2 = compute_Delta_d(prior_ranges,real_parameters1,RUN5_DATA[3])
print(f"Delta_d for RUN5 POINT 1097: {delta_d2}")

In [ ]:
# Create a plot of the Delta_d distribution for run 1 and run 5
# increase the font size for the plot
plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(figsize=(7, 5))
ax.hist(delta_1_dist, bins=30, color='g', histtype='step', density=True, label='MCMC')
ax.hist(delta_5_dist, bins=30, color='magenta', histtype='step', density=True, label='PTLMC')
ax.hist(delta_5_dist_moreT, bins=30, color='b', histtype='step', density=True, label='PTLMC, more T')
ax.hist(delta_5_dist_moreTP, bins=30, color='orange', histtype='step', density=True, label='PTLMC, more T, more P')
ax.set_xlabel(r'$\Delta$')
ax.set_ylabel(r'$p(\Delta)$')
ax.legend()
ax.set_yscale('log')
ax.set_xscale('log')
plt.tight_layout()
plt.savefig('Delta_hist_RUN1_RUN5.pdf')

In [ ]:
extract_parameters(RUN1_DATA[3])

In [ ]:
extract_parameters(RUN2_DATA[3])

In [ ]:
extract_parameters(RUN3_DATA[3])

In [ ]:
extract_parameters(RUN5_MORE_TP_DATA[3])

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data_1095/example_data_test_point1099.pkl')
for i in range(len(real_parameters)):
    print(f"{PARAMETER_NAMES[i]}: {real_parameters[i]:.3f}")

## Plot functions for the parametrized curves

In [ ]:
def parametrization_eta_over_s_vs_mu_B(eta_0, eta_2, eta_4, mu_B):
    eta_over_s = np.zeros_like(mu_B)  # Create an array to store the results

    # Compute eta_over_s for different ranges of mu_B
    eta_over_s[(mu_B >= 0) & (mu_B <= 0.2)] = eta_0 + (eta_2 - eta_0) * (mu_B[(mu_B >= 0) & (mu_B <= 0.2)] / 0.2)
    eta_over_s[(mu_B > 0.2) & (mu_B < 0.4)] = eta_2 + (eta_4 - eta_2) * ((mu_B[(mu_B > 0.2) & (mu_B < 0.4)] - 0.2) / 0.2)
    eta_over_s[mu_B >= 0.4] = eta_4

    return eta_over_s

def plot_eta_s_posterior(true_parameters,chain_parameters,filename):
    mu_B = np.linspace(0.,0.6,100)
    eta_s_parameters_true = true_parameters[12:15]
    true_eta_s = parametrization_eta_over_s_vs_mu_B(eta_s_parameters_true[0],eta_s_parameters_true[1],eta_s_parameters_true[2],mu_B)

    chain_array = chain_parameters[3]
    samples = chain_array[:,:,:].reshape((-1,chain_array.shape[2]))
    samples_eta_s = samples[:,12:15]

    posterior_eta_s = []
    for s in range(samples_eta_s.shape[0]):
        eta0 = samples_eta_s[s,0]
        eta2 = samples_eta_s[s,1]
        eta4 = samples_eta_s[s,2]
        posterior_eta_s.append(parametrization_eta_over_s_vs_mu_B(eta0,eta2,eta4,mu_B))
    posterior_eta_s = np.array(posterior_eta_s)
    median = np.percentile(posterior_eta_s, 50, axis=0)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,4),
                         sharex=False, sharey=False, constrained_layout=True)
    ax.plot(mu_B, true_eta_s, 'k--', label='Truth', lw=2)
    ax.plot(mu_B, median, 'g-', label="Marginalzied median", lw=2)

    for CL in [68,95,99.7]:
        low, high = np.percentile(posterior_eta_s, [50-CL/2., 50+CL/2.], axis=0)
        ax.fill_between(mu_B, low, high, alpha=0.2, color='g')
        ax.annotate("{}%CL".format(CL), xy=(mu_B[-1]*0.85, low[-1]), va="center", ha="left")

    ax.legend(loc='upper left')
    ax.set_xlabel('$\mu_B$ [GeV]')
    ax.set_ylabel('$\eta/s$')
    ax.set_xlim([0.,0.6])
    plt.savefig(f'{filename}.pdf')

In [ ]:
plot_eta_s_posterior(real_parameters,RUN1_DATA,'eta_s_posterior_RUN1')
plot_eta_s_posterior(real_parameters,RUN2_DATA,'eta_s_posterior_RUN2')
plot_eta_s_posterior(real_parameters,RUN3_DATA,'eta_s_posterior_RUN3')
plot_eta_s_posterior(real_parameters,RUN5_MORE_TP_DATA,'eta_s_posterior_RUN5')
plot_eta_s_posterior(real_parameters,RUN6_DATA,'eta_s_posterior_RUN6')
plot_eta_s_posterior(real_parameters,RUN7_DATA,'eta_s_posterior_RUN7')
plot_eta_s_posterior(real_parameters,RUN8_DATA,'eta_s_posterior_RUN8')

In [ ]:
def parametrization_zeta_over_s_vs_T(zeta_max, T_zeta0, sigma_plus, sigma_minus, T, mu_B):
    T_zeta_muB = T_zeta0 - 0.15 * mu_B**2.

    # Compute zeta_over_s for different conditions using NumPy's vectorized operations
    zeta_over_s = np.where(T < T_zeta0,
                           zeta_max * np.exp(-(T - T_zeta_muB)**2. / (2. * sigma_minus**2.)),
                           zeta_max * np.exp(-(T - T_zeta_muB)**2. / (2. * sigma_plus**2.)))

    return zeta_over_s

def plot_zeta_s_posterior(true_parameters,chain_parameters,filename):
    T = np.linspace(0.,0.5,100)
    zeta_s_parameters_true = true_parameters[15:19]
    true_zeta_s = parametrization_zeta_over_s_vs_T(zeta_s_parameters_true[0],zeta_s_parameters_true[1],zeta_s_parameters_true[2],zeta_s_parameters_true[3],T,0.)

    chain_array = chain_parameters[3]
    samples = chain_array[:,:,:].reshape((-1,chain_array.shape[2]))
    samples_zeta_s = samples[:,15:19]

    posterior_zeta_s = []
    for s in range(samples_zeta_s.shape[0]):
        zeta_max = samples_zeta_s[s,0]
        T_zeta0 = samples_zeta_s[s,1]
        sigma_plus = samples_zeta_s[s,2]
        sigma_minus = samples_zeta_s[s,3]
        posterior_zeta_s.append(parametrization_zeta_over_s_vs_T(zeta_max,T_zeta0,sigma_plus,sigma_minus,T,0.))
    posterior_zeta_s = np.array(posterior_zeta_s)
    median = np.percentile(posterior_zeta_s, 50, axis=0)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,4),
                         sharex=False, sharey=False, constrained_layout=True)
    ax.plot(T, true_zeta_s, 'k--', label='Truth', lw=2)
    ax.plot(T, median, 'g-', label="Marginalzied median", lw=2)

    for CL in [68,95,99.7]:
        low, high = np.percentile(posterior_zeta_s, [50-CL/2., 50+CL/2.], axis=0)
        ax.fill_between(T, low, high, alpha=0.2, color='g')
        ax.annotate("{}%CL".format(CL), xy=(T[-1]*0.85, high[-1]), va="center", ha="left")

    ax.legend(loc='upper left')
    ax.set_xlabel('$T$ [GeV]')
    ax.set_ylabel('$\zeta/s$')
    ax.set_xlim([0.,0.5])
    plt.savefig(f'{filename}.pdf')

In [ ]:
plot_zeta_s_posterior(real_parameters,RUN1_DATA,'zeta_s_posterior_RUN1')
plot_zeta_s_posterior(real_parameters,RUN2_DATA,'zeta_s_posterior_RUN2')
plot_zeta_s_posterior(real_parameters,RUN3_DATA,'zeta_s_posterior_RUN3')
plot_zeta_s_posterior(real_parameters,RUN5_MORE_TP_DATA,'zeta_s_posterior_RUN5')
plot_zeta_s_posterior(real_parameters,RUN6_DATA,'zeta_s_posterior_RUN6')
plot_zeta_s_posterior(real_parameters,RUN7_DATA,'zeta_s_posterior_RUN7')
plot_zeta_s_posterior(real_parameters,RUN8_DATA,'zeta_s_posterior_RUN8')

In [ ]:
def parametrization_y_loss_vs_y_init(yloss_2, yloss_4, yloss_6, y_init):
    # Create arrays to store the results
    y_loss = np.zeros_like(y_init)

    # Compute y_loss for different ranges of y_init using NumPy's vectorized operations
    y_loss[(y_init >= 0) & (y_init <= 2)] = yloss_2 * (y_init[(y_init >= 0) & (y_init <= 2)] / 2.)
    y_loss[(y_init > 2) & (y_init < 4)] = yloss_2 + (yloss_4 - yloss_2) * ((y_init[(y_init > 2) & (y_init < 4)] - 2.) / 2.)
    y_loss[y_init >= 4] = yloss_4 + (yloss_6 - yloss_4) * ((y_init[y_init >= 4] - 4.) / 2.)

    return y_loss

def plot_yloss_posterior(true_parameters,chain_parameters,filename):
    yinit = np.linspace(0.,6.2,100)
    yloss_parameters_true = true_parameters[2:5]
    true_yloss = parametrization_y_loss_vs_y_init(yloss_parameters_true[0],yloss_parameters_true[1],yloss_parameters_true[2],yinit)

    chain_array = chain_parameters[3]
    samples = chain_array[:,:,:].reshape((-1,chain_array.shape[2]))
    samples_yloss = samples[:,2:5]

    posterior_yloss = []
    for s in range(samples_yloss.shape[0]):
        yloss2 = samples_yloss[s,0]
        yloss4 = samples_yloss[s,1]
        yloss6 = samples_yloss[s,2]
        posterior_yloss.append(parametrization_y_loss_vs_y_init(yloss2,yloss4,yloss6,yinit))
    posterior_yloss = np.array(posterior_yloss)
    median = np.percentile(posterior_yloss, 50, axis=0)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,4),
                         sharex=False, sharey=False, constrained_layout=True)
    ax.plot(yinit, true_yloss, 'k--', label='Truth', lw=2)
    ax.plot(yinit, median, 'g-', label="Marginalzied median", lw=2)

    for CL in [68,95,99.7]:
        low, high = np.percentile(posterior_yloss, [50-CL/2., 50+CL/2.], axis=0)
        ax.fill_between(yinit, low, high, alpha=0.2, color='g')
        ax.annotate("{}%CL".format(CL), xy=(yinit[-1]*0.85, high[-1]), va="center", ha="left")

    # Fill the area above the line y = x with gray
    ax.fill_between(yinit, yinit, 4.5, where=(yinit <= 4.5), color='gray', alpha=0.2)

    ax.legend(loc='upper left')
    ax.set_xlabel('$y_{\mathrm{init}}$')
    ax.set_ylabel('$\langle y_{\mathrm{loss}}\\rangle$')
    ax.set_xlim([0.,6.2])
    plt.savefig(f'{filename}.pdf')

In [ ]:
plot_yloss_posterior(real_parameters,RUN1_DATA,'yloss_posterior_RUN1')
plot_yloss_posterior(real_parameters,RUN2_DATA,'yloss_posterior_RUN2')
plot_yloss_posterior(real_parameters,RUN3_DATA,'yloss_posterior_RUN3')
plot_yloss_posterior(real_parameters,RUN5_MORE_TP_DATA,'yloss_posterior_RUN5')
plot_yloss_posterior(real_parameters,RUN6_DATA,'yloss_posterior_RUN6')
plot_yloss_posterior(real_parameters,RUN7_DATA,'yloss_posterior_RUN7')
plot_yloss_posterior(real_parameters,RUN8_DATA,'yloss_posterior_RUN8')